# Nuggets creation and scoring

In [1]:
from pyterrier_rag.backend import OpenAIBackend
from transformers import AutoTokenizer
import os

model_name = "llama-3.3-70b-instruct"
#model_name = "qwen-2.5-72b-instruct"

tokenizer = AutoTokenizer.from_pretrained("casperhansen/llama-3.3-70b-instruct-awq")
#tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-72B-Instruct-AWQ")

generation_args={
    "temperature": 0.6,
    "max_tokens": 256,
}

# this could equally be a real OpenAI models
backend = OpenAIBackend(model_name, 
                      api_key=os.environ['IDA_LLM_API_KEY'],
                      generation_args=generation_args,
                      base_url="http://api.llm.apps.os.dcs.gla.ac.uk/v1", 
                      verbose=True, 
                      parallel=64)

/opt/miniconda3/envs/emalir-rq3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

def save_csv(path, content):
    content.to_csv(path, index=False)

def load_csv(path):
    try:
        content = pd.read_csv(path)
        return content
    except Exception:
        return None

In [3]:
from open_nuggetizer.nuggetizer import Nuggetizer
from open_nuggetizer._types import NuggetAssignMode
from fastchat.model import get_conversation_template

conv_template = get_conversation_template("meta-llama-3.1-sp")

nuggetizer = Nuggetizer(
    backend=backend, 
    conversation_template=conv_template,
    verbose=True,
    assigner_mode=NuggetAssignMode.SUPPORT_GRADE_3
)

#nuggets = load_csv("../data/processed/rag/nuggets.jsonl")
#if nuggets is None:
#    nuggets = nuggetizer.create(baseline)
#    save_csv("nuggets.csv", nuggets)

scored_nuggets = load_csv("../data/processed/rag/scored_nuggets.csv")
if scored_nuggets is None:
    scored_nuggets = nuggetizer.score(nuggets)
    save_csv("scored_nuggets.csv", scored_nuggets)

# Evaluation

In [4]:
import pyterrier as pt

print("🔍 Carico ranking esistente...")
df_run_base = pd.read_csv(
    "../data/processed/rag/__setencoder-novelty-base__msmarco-segment-trecrag24.tsv", sep="\t",
    names=["qid", "Q0", "docno", "rank", "score", "run_name"]
)
df_run_ea = pd.read_csv(
    "../data/processed/rag/__setencoder-novelty-ea__msmarco-segment-trecrag24.tsv", sep="\t",
    names=["qid", "Q0", "docno", "rank", "score", "run_name"]
)

df_queries = pd.read_csv("../data/raw/rag/trecrag24-queries.tsv", sep="\t", names=["qid", "query"])
df_queries = df_queries[df_queries["qid"].isin(scored_nuggets["qid"])]

def get_rank(df_queries):
    run = df_queries.merge(df_run_ea, on="qid", how="left")
    return run
get_rank_pipe = pt.apply.generic(get_rank)

def rename_segment(run):
    run = run.rename(columns={"segment": "text"})
    return run
rename_pipe = pt.apply.generic(rename_segment)

🔍 Carico ranking esistente...


In [5]:
import ir_datasets 
dataset = ir_datasets.load('msmarco-segment-v2.1')
pt_dataset = pt.get_dataset("irds:msmarco-segment-v2.1")

In [6]:
import pyterrier_alpha as pta
index = pta.Artifact.from_hf('namawho/msmarco-segment-v2.1.pisa')

In [7]:
from pyterrier_rag.prompt import Concatenator
from pyterrier_rag.readers import Reader
from pyterrier_rag.prompt import PromptTransformer
from jinja2 import Template

def make_callable_template(template: Template):
    def template_call(**kwargs):
        return template.render(**kwargs)

    return template_call

GENERIC_PROMPT = Template(
    "Use the context information to answer the Question: \n Context: {{ context }} \n Question: {{ query }} \n Answer:"
)

prompt = PromptTransformer(
            instruction=make_callable_template(GENERIC_PROMPT),
            system_message="You are an helpful assistant.",
            conversation_template=conv_template,
            input_fields=[
                "qcontext",
                "query",
            ],
        )

reader = Reader(backend, prompt)
rag_pipeline = get_rank_pipe % 10 >>  pt.text.get_text(pt_dataset, "segment") >> rename_pipe >> Concatenator() >> reader

In [102]:
results = (rag_pipeline)(df_queries)
results

,prompt,qid,query_0,qanswer
0,"[{'role': 'system', 'content': 'You are Nugget...",2024-145979,what is vicarious trauma and how can it be cop...,"Vicarious trauma, also known as secondary trau..."
1,"[{'role': 'system', 'content': 'You are Nugget...",2024-216592,why disability insurance is a smart investment,Disability insurance is a smart investment for...
2,"[{'role': 'system', 'content': 'You are Nugget...",2024-32912,how bad did the vietnam war devastate the econ...,The Vietnam War had a significant impact on th...
3,"[{'role': 'system', 'content': 'You are Nugget...",2024-153051,what target stors's policies for shoplifting,"Based on the context, it appears that Target s..."
4,"[{'role': 'system', 'content': 'You are Nugget...",2024-79081,how taylor swift's age affects her relationships,Based on the context of Taylor Swift's life an...
5,"[{'role': 'system', 'content': 'You are Nugget...",2024-42497,how does the informal recycling sector add to ...,The informal recycling sector contributes to a...
6,"[{'role': 'system', 'content': 'You are Nugget...",2024-158743,what was happening in germany and netherlands ...,"In the 1840s, several significant events were ..."
7,"[{'role': 'system', 'content': 'You are Nugget...",2024-143869,what is scientific evidence for or against the...,There is scientific evidence both for and agai...
8,"[{'role': 'system', 'content': 'You are Nugget...",2024-43037,how europe solved its russian gas problem,To address how Europe solved its Russian gas p...
9,"[{'role': 'system', 'content': 'You are Nugget...",2024-43983,how has inclusivity made vogue magazine more p...,"In recent years, Vogue magazine has made a con..."


In [142]:
scored_nuggets = scored_nuggets.rename(columns={"query_id": "qid"})
results = results.rename(columns={"query_id": "qid", "query_0": "query"})
scores = []

for element in nuggetizer.AllScore(strict=True).iter_calc(scored_nuggets, results):
    print(f"Query ID: {element.query_id}, Measure: {element.measure}, Value: {element.value}")
    scores.append(element.value)



pt.apply.by_query():   0%|          | 0/779 [00:00<?, ?it/s]


  0%|          | 0/2 [00:00<?, ?window/s]


 50%|█████     | 1/2 [00:02<00:02,  2.65s/window]


100%|██████████| 2/2 [00:05<00:00,  2.95s/window]


pt.apply.by_query():   2%|▏         | 16/779 [00:05<04:41,  2.71it/s]

Assignments: [0, 1, 1, 2, 0, 2, 0, 2, 2, 0, 0, 1, 1, 1, 2]





  0%|          | 0/2 [00:00<?, ?window/s]


 50%|█████     | 1/2 [00:03<00:03,  3.22s/window]


100%|██████████| 2/2 [00:06<00:00,  3.00s/window]


pt.apply.by_query():   5%|▍         | 36/779 [00:11<04:01,  3.08it/s]

Assignments: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 2, 0, 0]





  0%|          | 0/1 [00:00<?, ?window/s]


100%|██████████| 1/1 [00:03<00:00,  3.07s/window]


pt.apply.by_query():   6%|▌         | 45/779 [00:14<04:01,  3.03it/s]

...

Assignments: [0, 0, 1, 0, 0, 0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 0]





  0%|          | 0/2 [00:00<?, ?window/s]


 50%|█████     | 1/2 [00:02<00:02,  2.82s/window]


100%|██████████| 2/2 [00:05<00:00,  2.95s/window]


pt.apply.by_query(): 100%|██████████| 779/779 [04:16<00:00,  3.04it/s]


Assignments: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]




  0%|          | 0/2 [00:00<?, ?window/s]

 50%|█████     | 1/2 [00:01<00:01,  1.12s/window]

100%|██████████| 2/2 [00:04<00:00,  2.06s/window]

Assignments: [0, 0, 2, 2, 2, 2, 0, 0, 1, 0, 0, 1]
Query ID: 2024-145979, Measure: AllScore(strict=True), Value: 0.3333333333333333
Query ID: 2024-216592, Measure: AllScore(strict=True), Value: 0.1
Query ID: 2024-32912, Measure: AllScore(strict=True), Value: 0.2222222222222222
Query ID: 2024-153051, Measure: AllScore(strict=True), Value: 0.21428571428571427
Query ID: 2024-79081, Measure: AllScore(strict=True), Value: 0.25
Query ID: 2024-42497, Measure: AllScore(strict=True), Value: 0.3333333333333333
Query ID: 2024-158743, Measure: AllScore(strict=True), Value: 0.1111111111111111
Query ID: 2024-143869, Measure: AllScore(strict=True), Value: 0.2
Query ID: 2024-43037, Measure: AllScore(strict=True), Value: 0.4
Query ID: 2024-43983, Measure: AllScore(strict=True), Value: 0.0
Query ID: 2024-224279, Measure: AllScore(strict=True), Value: 0.26666666666666666
Query ID: 2024-214096, Measure: AllScore(strict=True), Value: 0.1875
Query ID: 2024-219624, Measure: AllScore(strict=True), Value: 0.538

In [143]:
import numpy as np
# Calcolo della media (e deviazione standard, se ti serve)
mean_vital = np.mean(scores)
std_vital = np.std(scores)
print(f"\n✅ Score medio: {mean_vital:.4f} (± {std_vital:.4f})")


✅ Score medio: 0.2540 (± 0.1709)
